In [39]:
import numpy as np
import matplotlib.pyplot as plt
import pylcp
from pylcp.atom import atom
import scipy.constants as cts
import magpylib as magpy
from scipy.stats import rv_continuous

In [ ]:
tfmot = magpy.Collection()
N = 0
angles = [0,90,180,270]
angles_d = [45,225,225,45]
l = 5
p = [0,0,0,0,20,20,20,20]
s  = [-1,-1,1,1,1,1,-1,-1]
H = 40
h = [-H,-H,-H,-H,H,H,H,H]
mstyle = dict(
    mode="color+arrow",
    color=dict(north="magenta", middle="white", south="turquoise"),
    arrow=dict(width=2, color="k")
)
for a in range(len(angles)):
    cube= magpy.magnet.Cuboid(
    dimension=(6,8,80),
    polarization=(s[a]*1.42,0,0),
    position=(h[a],h[a],h[a]),
    style_magnetization=mstyle )
    
    cube.rotate_from_angax(angles[a], 'z',anchor = 0)
    cube.rotate_from_angax(angles_d[a], 'z')

    tfmot.add(cube)

magpy.show(tfmot.rotate_from_angax(45,'z'), backend='plotly')

In [ ]:
# Continuation from above - ensure previous code is executed

import matplotlib.pyplot as plt
fig, ax = plt.subplots()

# Compute and plot field on x-y grid
grid = np.mgrid[-10:10:100j, -10:10:100j, 0:0:1j].T[0]
X, Y, _ = np.moveaxis(grid, 2, 0)

B = tfmot.getB(grid)*1e3
Bx, By, _ = np.moveaxis(B, 2, 0)
Bamp = np.linalg.norm(B, axis=2)

pc = ax.contourf(X, Y, Bamp, levels=50, cmap="coolwarm")
ax.streamplot(X, Y, Bx, By, color="k", density=1.5, linewidth=1)

# Add colorbar
fig.colorbar(pc, ax=ax, label="|B| (mT)")

# Figure styling
ax.set(
    xlabel="x-position(mm)",
    ylabel="y-position(mm)",
    aspect=1,
)

plt.show()

In [ ]:
rb87 = atom('87Rb')

klab = 2*np.pi*rb87.transition[1].k # Lab wavevector (without 2pi) in cm^{-1}
taulab = rb87.state[2].tau  # Lifetime of 6P_{3/2} state (in seconds)
gammalab = 1/taulab
Blab = 15 # About 15 G/cm is a typical gradient for Rb
mass_lab = 173*cts.value('atomic mass constant')
print(klab, taulab, gammalab/2/np.pi)

In [ ]:
# Now, here are our `natural' length and time scales:
x0 = 1/klab  # cm
t0 = taulab  # s
vc = x0/t0*(1/100)
Fc = gammalab*cts.hbar*klab*100


mass = 87*cts.value('atomic mass constant')*(x0*1e-2)**2/cts.hbar/t0

# And now our wavevector, decay rate, and magnetic field gradient in these units:
k = klab*x0
gamma = gammalab*t0
alpha = cts.value('Bohr magneton')*1e-4*5*x0*t0/cts.hbar

print(x0, t0, mass, k, gamma, alpha)

In [ ]:
H_g_D2, mu_q_g_D2 = pylcp.hamiltonians.hyperfine_coupled(
0, 5/2, 0,  	-0.2592,
    Ahfs = 0, Bhfs=0, Chfs=0,
    muB=1)# ground state 1s0
H_e_D2, mu_q_e_D2 = pylcp.hamiltonians.hyperfine_coupled(
1, 5/2, 1.035,	-0.2592,
    Ahfs=59.52*1e6,Bhfs = 601.87*1e6 , Chfs= 0,
    muB=1) #excited state 1p1

dijq_D2 = pylcp.hamiltonians.dqij_two_hyperfine_manifolds(0, 1, 5/2)

E_e_D2 = np.unique(np.diagonal(H_e_D2))
E_g_D2 = np.unique(np.diagonal(H_g_D2))
hamiltonian_D2 = pylcp.hamiltonian(mass = mass)
hamiltonian_D2.add_H_0_block('g', H_g_D2)
hamiltonian_D2.add_H_0_block('e', H_e_D2)
hamiltonian_D2.add_d_q_block('g', 'e', dijq_D2, gamma = gamma, k = k)
hamiltonian_D2.add_mu_q_block('g', mu_q_g_D2)
hamiltonian_D2.add_mu_q_block('e', mu_q_e_D2)



dijq = pylcp.hamiltonians.dqij_two_bare_hyperfine(0, 1)




det = -1
s = 1.5

# Define the laser beams:
laserBeams = pylcp.laserBeams(
    [{'kvec':k*np.array([ 1.,  0., 0.]), 's':s, 'pol':-1, 'delta': E_e_D2[2] - E_g_D2[0] +  det*gamma},
     {'kvec':k*np.array([-1.,  0., 0.]), 's':s, 'pol':-1, 'delta': E_e_D2[2] - E_g_D2[0] +  det*gamma},
     {'kvec':k*np.array([ 0.,  1,  0.]), 's':s, 'pol':1, 'delta': E_e_D2[2] - E_g_D2[0] +  det*gamma},
     {'kvec':k*np.array([ 0., -1., 0.]), 's':s, 'pol':1, 'delta': E_e_D2[2] - E_g_D2[0] +  det*gamma}],
beam_type= pylcp.infinitePlaneWaveBeam
)

# Define the magnetic field:
print(alpha)
bf_mot = lambda R:tfmot.getB(R*10*x0)*cts.value('Bohr magneton')*t0/cts.hbar 
#linGrad = pylcp.magField(lambda R: bf_mot(R))
linGrad = pylcp.magField(lambda R: -alpha*R)

In [60]:
rateeq = pylcp.rateeq(laserBeams, linGrad, hamiltonian_D2, include_mag_forces=True)

In [ ]:
x = np.arange(-5, 5, 0.04)/x0
v = np.arange(-14, 14, 0.4)/vc

X, V = np.meshgrid(x, v)

Rvec = np.array([ np.zeros(X.shape),X, np.zeros(X.shape)])
Vvec = np.array([np.zeros(V.shape),V, np.zeros(V.shape)])

rateeq.generate_force_profile(Rvec, Vvec, name='Fx', progress_bar=True)

In [ ]:

fig, ax = plt.subplots(nrows=1, ncols=1, num="Expression", figsize=(6.5, 2.75))
im1 = ax.contourf(np.array(x)*x0, np.array(v)*vc, rateeq.profile['Fx'].F[1]*Fc)
fig.subplots_adjust(left=0.08, wspace=0.2)
cb1 = plt.colorbar(im1)
cb1.set_label('$F(N)$')
ax.set_xlabel('$x$ (cm)')
ax.set_ylabel('$v (ms^{-1})$')
ax.set_title('Force profile (with real magnetic field)')


In [ ]:
# Constants
k_B = cts.k  # Boltzmann constant in J/K
 # Atomic mass of Yb-173
mass_kg = mass_lab  # Convert to kilograms
temperature = 405 + 273.15  # Convert Celsius to Kelvin

# Velocity Probability Function
def p(m, T, v):
    return (m / (k_B * T))**2 * np.exp(-m * v**2 / (2 * k_B * T)) * (v**3) / 2

In [ ]:
class MaxwellBoltzmann(rv_continuous):
    def _pdf(self, v):
        return p(mass_kg, temperature, v)

# Define the custom distribution
maxwell_boltzmann = MaxwellBoltzmann(a=0, name="Maxwell-Boltzmann")

In [ ]:
size = 100# Number of samples
velocities = maxwell_boltzmann.rvs(size=size)
velocities = velocities[velocities < 400]
velocities_t = [np.tan(35e-3)*i for i in velocities ]
v_0 = []
for i in range(len(velocities)):
    v_0.append([velocities_t[i],velocities_t[i],50])
# Display the sampled velocities
print(len(velocities))
plt.hist(velocities, bins=100, density=True)
plt.grid()

In [94]:
def simulate_trajectory(v_0, t_span, mass_lab):

    x_t, y_t, v_tx, v_ty, t = [], [], [], [], []

    di = 0  # Initial conditions

    s = np.array([0,0,0])
    
    b = 0

    for q in range(len(t_span)):

        v_x = v_0[0]

        v_y = v_0[1]

        v_z = v_0[2]


        #print(v_0)
        dt = (t_span[q] - di)/t0
        
        # if np.isnan(v_0).any():
        #     print(v_0 , s,dt,a_x)
        #     break

        b  = b +1

        #if any(v_0) < 1e-10:
         #   print(v_0)
          #  break
        print(np.array(v_0)*vc)
        if np.isnan(v_0).any():
            print(f'final b is {b}') 
            break
    
  
  
        rateeq.generate_force_profile(np.array(v_0), np.array(s), name='Fz', progress_bar=False)
     
        
        # Update position and velocity

        a_x = rateeq.profile['Fz'].F[0]/mass
        
        a_y = rateeq.profile['Fz'].F[1]/mass

        a_z = 0 
        if a_x*mass_lab*Fc < 1e-22 or np.isnan(a_x):
            a_x == 0

        if a_y*mass_lab*Fc < 1e-22 or np.isnan(a_y):
            a_y == 0
        s_x = (1 / 2) * a_x * (dt ** 2) + v_x * dt + s[0]

        s_y = (1 / 2) * a_y * (dt ** 2) + v_y * dt + s[1]

        s_z = (1 / 2) * a_z * (dt ** 2) + v_z * dt + s[2]
        #print(a_y)

        v_x = v_x + (a_x * dt)

        v_y = v_y + (a_y * dt)

        v_z = v_z + a_z*dt
        #print(v_x)
        # Append values
        x_t.append(s_x)
        y_t.append(s_y)
        v_tx.append(v_x)
        v_ty.append(v_y)
        t.append(t_span[q])

        di = t_span[q]
        v_0 = [v_x, v_y, v_z]
        s = [s_x, s_y, s_z]
        # print('sped')
        # print(v_0)
        # print('dis')
        # print(s)
        #print(dt)
    return x_t, y_t, v_tx, v_ty, t

In [ ]:
rateeq.generate_force_profile(np.array([13.61472577 ,13.61483196, 50. ])/vc,np.array([0.20442546, 0.20442679, 0.75075075])/x0,name = 'Fz')
print(rateeq.profile['Fz'].F[1])

In [ ]:
x_n = []
y_n = []
v_tx_n = []
v_ty_n = []
t_n = []
t_max = 0.05
t_span = np.linspace(0, t_max, 1000)
v_01 = [v_0[0]]
b = 0
for i in v_01:
    x_t, y_t, v_tx, v_ty, t = simulate_trajectory(np.array(i)/vc, t_span, mass_lab)

    x_n.append(x_t)
    y_n.append(y_t)
    v_tx_n.append(v_tx)
    v_ty_n.append(v_ty)
    t_n.append(t)

    b += 1
    print(b)